In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.8 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
from fastbook import *
from IPython.display import display, HTML

In [3]:
gdrive

Path('/content/gdrive/My Drive')

In [4]:
course_path = gdrive/'Colab Notebooks/fastbook/clean'
course_path

Path('/content/gdrive/My Drive/Colab Notebooks/fastbook/clean')

# NLP Deep Dive: RNNs

## Text Preprocessing

### Word Tokenization with fastai

In [5]:
from fastai.text.all import *

In [6]:
URLs.IMDB

'https://s3.amazonaws.com/fast-ai-nlp/imdb.tgz'

In [7]:
path = untar_data(URLs.IMDB)
path

Path('/root/.fastai/data/imdb')

In [8]:
files = get_text_files(path, folders=['train', 'test', 'unsup'])
files

(#100000) [Path('/root/.fastai/data/imdb/train/neg/10026_2.txt'),Path('/root/.fastai/data/imdb/train/neg/703_4.txt'),Path('/root/.fastai/data/imdb/train/neg/1041_1.txt'),Path('/root/.fastai/data/imdb/train/neg/5879_1.txt'),Path('/root/.fastai/data/imdb/train/neg/6649_1.txt'),Path('/root/.fastai/data/imdb/train/neg/8775_3.txt'),Path('/root/.fastai/data/imdb/train/neg/5304_2.txt'),Path('/root/.fastai/data/imdb/train/neg/7561_2.txt'),Path('/root/.fastai/data/imdb/train/neg/6231_4.txt'),Path('/root/.fastai/data/imdb/train/neg/5398_1.txt')...]

In [9]:
with open(files[0], 'r') as f:
    txt = f.read()
txt[:75]

'I had some expectation for the movie, since it had a nice star cast and it '

---

Demo of `coll_repr`:

In [10]:
x = list(range(10))
x

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
coll_repr(x, 3)

'(#10) [0,1,2...]'

In [12]:
coll_repr(x, 5)

'(#10) [0,1,2,3,4...]'

In [13]:
type(coll_repr(x, 5))

str

In [14]:
print(coll_repr(x, 5))

(#10) [0,1,2,3,4...]


---

In [15]:
spacy = WordTokenizer()
type(spacy)

fastai.text.core.SpacyTokenizer

In [16]:
type(spacy([txt]))

generator

**Note:** The generator generates a list of tokens for each string. If we pass a list with more than one string, then we'll get back more than one list.

In [17]:
help(first)

Help on function first in module fastcore.basics:

first(x, f=None, negate=False, **kwargs)
    First element of `x`, optionally filtered by `f`, or None if missing



In [18]:
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#272) ['I','had','some','expectation','for','the','movie',',','since','it','had','a','nice','star','cast','and','it','is','the','return','of','the','duo','of','Akshay','and','Saif','.','Well',','...]


In [19]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [20]:
tkn = Tokenizer(spacy)
type(tkn)

fastai.text.core.Tokenizer

In [21]:
type(tkn(txt))

fastcore.foundation.L

Notice two differences between `spacy` and `tkn`:

1. `spacy` accepts a collection of strings, whereas `tkn` accepts a single string.
2. `spacy` returns a generator, whereas `tkn` returns an `L` object.

In [22]:
print(coll_repr(tkn(txt), 31))

(#297) ['xxbos','i','had','some','expectation','for','the','movie',',','since','it','had','a','nice','star','cast','and','it','is','the','return','of','the','duo','of','xxmaj','akshay','and','xxmaj','saif','.'...]


In [23]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [24]:
??replace_rep

In [25]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### Subword Tokenization

In [26]:
txts = []
for o in files[:2000]:
    with open(o, 'r') as f:
        txts.append(f.read())
txts = L(txts)
txts

(#2000) ["I had some expectation for the movie, since it had a nice star cast and it is the return of the duo of Akshay and Saif. Well, I was hesitant to watch the movie because this was done by the same man who wrote the story for Dhoom franchise because I hated Dhoom 2; but if Dhoom 2 is compared to Tashan, I would say Dhoom 2 is very realistic. <br /><br />When I saw the credits at the beginning, I felt nice because it was put up in a nice way. Well, the very first scene itself pis*ed me off. Then, the major drawback of the movie is the action sequences. Me and my friends were laughing our guts off watching this crappy fights. It was like Akshay against some 30 thugs and all and the thugs even got machine guns! Phew...you got to see this to understand how bad the action sequences are.<br /><br />The other thing about the movie is the far too predictable story. It reminded me of some of the early 80's movies.<br /><br />Well, the only thing the movie is worth is of sexy Kareena, who 

In [27]:
sp = SubwordTokenizer(vocab_sz=1000)
type(sp)

fastai.text.core.SentencePieceTokenizer

Recall that SentencePiece is a common choice is multi-lingual models (such as XLMRoBERTa).

In [28]:
sp.setup(txts)

{'sp_model': Path('tmp/spm.model')}

**Note:** We're passing in a list of strings to `setup`.

In [29]:
type(sp([txt]))

generator

In [30]:
toks = first(sp([txt]))
print(coll_repr(toks, 40))

(#418) ['▁I','▁had','▁some','▁expect','ation','▁for','▁the','▁movie',',','▁since','▁it','▁had','▁a','▁nice','▁star','▁cast','▁and','▁it','▁is','▁the','▁re','t','ur','n','▁of','▁the','▁du','o','▁of','▁A','k','s','ha','y','▁and','▁S','a','if','.','▁Well'...]


In [31]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [32]:
subword(1000)

'▁I ▁had ▁some ▁expect ation ▁for ▁the ▁movie , ▁since ▁it ▁had ▁a ▁nice ▁star ▁cast ▁and ▁it ▁is ▁the ▁re t ur n ▁of ▁the ▁du o ▁of ▁A k s ha y ▁and ▁S a if . ▁Well'

In [33]:
subword(200)

'▁I ▁ h a d ▁some ▁ex p e c t a tion ▁for ▁the ▁movie , ▁s in ce ▁it ▁ h a d ▁a ▁ n ic e ▁s t ar ▁c a s t ▁and ▁it ▁is'

In [34]:
subword(10000)

'▁I ▁had ▁some ▁expect ation ▁for ▁the ▁movie , ▁since ▁it ▁had ▁a ▁nice ▁star ▁cast ▁and ▁it ▁is ▁the ▁return ▁of ▁the ▁duo ▁of ▁Akshay ▁and ▁Sa if . ▁Well , ▁I ▁was ▁he s it ant ▁to ▁watch'

### Numericalization with fastai

In [35]:
toks = tkn(txt)
print(coll_repr(toks, 31))

(#297) ['xxbos','i','had','some','expectation','for','the','movie',',','since','it','had','a','nice','star','cast','and','it','is','the','return','of','the','duo','of','xxmaj','akshay','and','xxmaj','saif','.'...]


In [36]:
# Reminder:
type(txts), len(txts)

(fastcore.foundation.L, 2000)

In [37]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#297) ['xxbos','i','had','some','expectation','for','the','movie',',','since'...]

In [38]:
num = Numericalize()
type(num)

fastai.text.data.Numericalize

**Note:** When instantiating `Numericalize`, we can specify `min_freq` and `max_vocab`. The reason is that the spaCy tokenizer doesn't create a vocab with a fixed number of tokens. It also doesn't set a minimum frequency. But SentencePiece has a fixed vocabulary size!

In [39]:
num.setup(toks200)

**Note:** We're passing in a list of lists of tokens to `setup`.

In [40]:
type(num.vocab), len(num.vocab)

(list, 2152)

The length of the vocab is less because we only used 200 texts to set it up.

In [41]:
coll_repr(num.vocab, 20)

"(#2152) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','it','i','this'...]"

In [42]:
nums = num(toks)[:20]
nums

TensorText([  2,  18,  77,  66,   0,  28,   9,  26,  10, 276,  17,  77,  13, 556, 335, 182,  12,  17,  16,   9])

In [43]:
' '.join(num.vocab[o] for o in nums)

'xxbos i had some xxunk for the movie , since it had a nice star cast and it is the'

### Putting Our Texts into Batches for a Language Model

In [44]:
# Just run:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [45]:
# Just run:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [46]:
# Just run:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [47]:
# Just run:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [48]:
nums200 = toks200.map(num)
type(nums200)

fastcore.foundation.L

In [49]:
len(nums200)

200

In [50]:
nums200[0]

TensorText([   2,   18,   77,   66,    0,   28,    9,   26,   10,  276,   17,   77,   13,  556,  335,  182,   12,   17,   16,    9,  682,   14,    9,    0,   14,    8, 1255,   12,    8,    0,   11,
               8,  110,   10,   18,   25,    0,   15,  132,    9,   26,   91,   19,   25,  199,   48,    9,  170,  163,   61,  759,    9,  115,   28,    8, 1256, 1556,   91,   18, 1557,    8, 1256,
             192,  123,   29,   59,    8, 1256,  192,   16, 1035,   15,    8,    0,   10,   18,   65,  179,    8, 1256,  192,   16,   83,    0,   11,   27,    8,   75,   18,  205,    9,  463,   45,
               9,  307,   10,   18,  388,  556,   91,   17,   25,  206,   69,   20,   13,  556,  130,   11,    8,  110,   10,    9,   83,   95,  156,  683,    0,   89,  117,   11,    8,  107,   10,
               9,  684,    0,   14,    9,   26,   16,    9,  298,  877,   11,    8,   89,   12,   85,  506,   70,  878,  507,    0,  117,  168,   19, 1558, 1559,   11,    8,   17,   25,   54,    8,
          

In [52]:
nums200[0].shape

torch.Size([297])

In [53]:
nums200[1]

TensorText([   2,    8,  686, 1036,   10,   61,   41,    0,   58,   13, 1561,    0,   10,   41,    0,   48,   52,    0,  224,   11,   27,    8,    9,    8,    0,    8,   21,    8,    0,    8,  607,
              37,    0,    8,    0,   35,   16,   13,  299,   14,   13,    0,    0,   28,  881,  882,   11,    8,    9,  128,  389, 1037,   41,  150,  267,   12,    9,  115,   28,    9,  116,  209,
              16,  150,    0,   10,   51,   23,   68,   13,  687,    0, 1562,  225,   28,   13,  608,    0,   11,    8,   20,  249,   92,   14,    9,  115,   23, 1563,   24,   69,   16,  150, 1564,
              10,  435,   28,   42,   43,  121, 1565,  883,   11,    8,   29,    9,   26,   16,   90,  114,  161,   48,  141, 1566,    0,    0,   37,   20,   52,    0, 1567,   35,   12,   42, 1568,
               0,   10,    0,  884,   11,   27,    8,    9,  182,   16, 1569,   28,    9,  116,  209,   11,    8, 1570,    8,    0,   16,    9,  230,   14,    9,  238,   33,   50,  104,   13,  150,
          

In [54]:
nums200[1].shape

torch.Size([220])

In [51]:
dl = LMDataLoader(nums200)
type(dl)

fastai.text.data.LMDataLoader

In [55]:
x, y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [56]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos i had some xxunk for the movie , since it had a nice star cast and it is the'

In [57]:
' '.join(num.vocab[o] for o in y[0][:20])

'i had some xxunk for the movie , since it had a nice star cast and it is the return'

## Training a Text Classifier

### Language Model Using DataBlock

In [55]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb,
    splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [56]:
type(dls_lm)

fastai.data.core.DataLoaders

In [57]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos i was never in the past interested in this play although love xxmaj shakespeare and have seen most of his plays now and enthusiastically studied some at school . xxmaj something about this story and all the fuss about it seemed to put me off . i never bothered to try to see xxmaj hamlet until fairly recently deciding i should at least try to watch it and i borrowed the xxmaj olivier version from the library . xxmaj,i was never in the past interested in this play although love xxmaj shakespeare and have seen most of his plays now and enthusiastically studied some at school . xxmaj something about this story and all the fuss about it seemed to put me off . i never bothered to try to see xxmaj hamlet until fairly recently deciding i should at least try to watch it and i borrowed the xxmaj olivier version from the library . xxmaj well
1,""" life "" then the pro - capital punishment lobby would not have a leg to stand on . xxmaj but it does n't . xxmaj and so they do . \n\n xxmaj there were times when xxmaj aileen came across as likable . xxmaj genuinely likable . xxmaj and one could fully understand why the film maker felt a rapport with his subject . xxmaj when she was lucid she made sense . xxmaj she knew she had been","life "" then the pro - capital punishment lobby would not have a leg to stand on . xxmaj but it does n't . xxmaj and so they do . \n\n xxmaj there were times when xxmaj aileen came across as likable . xxmaj genuinely likable . xxmaj and one could fully understand why the film maker felt a rapport with his subject . xxmaj when she was lucid she made sense . xxmaj she knew she had been stitched"


Let's explore an actual minibatch comprising input IDs.

In [58]:
dls_lm[0]

In [59]:
x_batch, y_batch = next(iter(dls_lm[0]))
x_batch.shape, y_batch.shape

(torch.Size([128, 80]), torch.Size([128, 80]))

In [60]:
x_batch[0]

LMTensorText([    2,     8,   129,    13,  6014,  1823,   564,    11,    19,    85,   527,    20,    30,    11,   263,    70,    52,    94,     9,   849,   420,    10,     8,    44,    14,     9,
               1563,   404,    18,     9,    30,    16,    69,     9, 28936,    11,    50,    87,  1855,    41,     8,  1576,  3102,    11,    16,  6210,    37,   195,    34,    48,     8,  3282,
                318,    13,  6014,     0,    37,    49,  1635,   136,    13,   601,  1823,   312,  4840,   467,   457,  1810,    46,   302,    34,    10,     8,    40,   492,    52,  6735,    21,
                 40,   241], device='cuda:0')

In [61]:
y_batch[0]

TensorText([    8,   129,    13,  6014,  1823,   564,    11,    19,    85,   527,    20,    30,    11,   263,    70,    52,    94,     9,   849,   420,    10,     8,    44,    14,     9,  1563,   404,
               18,     9,    30,    16,    69,     9, 28936,    11,    50,    87,  1855,    41,     8,  1576,  3102,    11,    16,  6210,    37,   195,    34,    48,     8,  3282,   318,    13,  6014,
                0,    37,    49,  1635,   136,    13,   601,  1823,   312,  4840,   467,   457,  1810,    46,   302,    34,    10,     8,    40,   492,    52,  6735,    21,    40,   241,     0],
           device='cuda:0')

In [62]:
type(dls_lm.vocab), len(dls_lm.vocab)

(list, 60008)

In [64]:
print(dls_lm.vocab[:30])

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', 'the', '.', ',', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', '"', "'s", '-', 'was', '\n\n', 'as', 'with', 'for']


### Fine-Tuning the Language Model

In [65]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()]).to_fp16()

In [66]:
type(learn)

fastai.text.learner.LMLearner

In [67]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.004985,3.907380,0.300211,49.768383,13:20


### Saving and Loading Models

In [68]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [69]:
learn = learn.load('1epoch')

In [70]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.751968,3.769070,0.315992,43.339737,13:49
1,3.703485,3.708263,0.323104,40.782921,13:33
2,3.646224,3.659706,0.328522,38.849907,13:50
3,3.539139,3.624274,0.332636,37.497475,13:51
4,3.497658,3.602425,0.335232,36.687096,13:31
5,3.430049,3.587172,0.337779,36.131752,13:33
6,3.350357,3.578308,0.339165,35.812908,13:40
7,3.304235,3.574070,0.340153,35.661457,14:03
8,3.257838,3.576935,0.340398,35.763752,14:18
9,3.212648,3.581804,0.340225,35.938301,13:56


In [71]:
learn.save_encoder('finetuned')

### Text Generation

In [72]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]

In [73]:
preds

["i liked this movie because it dealt with real issues which were very important . It was good for the movie 's part and for that , i think the performances by Gary Cooper , Helen Hunt , and",
 "i liked this movie because it was a good comedy . The two lead actors are really good . It could have been a real comedy . If you can , if you 're a fan of the Comedy Central"]

In [74]:
print("\n".join(preds))

i liked this movie because it dealt with real issues which were very important . It was good for the movie 's part and for that , i think the performances by Gary Cooper , Helen Hunt , and
i liked this movie because it was a good comedy . The two lead actors are really good . It could have been a real comedy . If you can , if you 're a fan of the Comedy Central


### Creating the Classifier DataLoaders

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

## Disinformation and Language Models

## Conclusion

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?

### Further Research

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?